In [1]:
import torch
from torchmetrics import PearsonCorrCoef
from module.metircs import R2

In [20]:
from torchmetrics import R2Score
target = torch.tensor([3, -0.5, 2, 7])
preds = torch.tensor([2.5, 0.0, 2, 8])
r2score = R2Score(multioutput='variance_weighted', adjusted=0)
r2score(preds, target)

tensor(0.9486)

In [7]:
x = torch.tensor([[3, -0.5, 2, 7]])
y = torch.tensor([[2.5, 0.0, 2, 8]])

In [8]:
pearsonr = R2(num_targets=1, summarize=False)

In [9]:
pearsonr.update(x, y)

In [10]:
pearsonr.compute()

tensor(0.9613)

In [11]:
pearsonr.reset()